In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

from grad import norm_of_surface
from vector import Vec
from intercept import within_error, estimate_all_intercepts, nearest_point, is_in_interval

In [2]:
# estimate_all_intercepts(f, g, interval, n_tests = 100, n_iter = 100, error = 1e-10):

N_TESTS = 1000
N_ITER = 100
ERROR = 1e-10
CLIP_DIST = 1e-9

In [3]:
def sign(num):
    val = np.sign(num)
    if val != 0:
        return val
    else:
        return 1

def reflect(inc: Vec, norm: Vec) -> Vec:
    if not isinstance(inc, Vec):
        raise TypeError("inc must be a Vec type")
    if not isinstance(norm, Vec):
        raise TypeError("norm must be a Vec type")
    
    inc.normalise()
    norm.normalise()
    
    A = inc.dot(norm)
    return inc.add(-norm.times(2 * A)).normalise()

def refract(inc: Vec, norm: Vec, n1, n2) -> Vec:
    if not isinstance(inc, Vec):
        raise TypeError("inc must be a Vec type")
    if not isinstance(norm, Vec):
        raise TypeError("norm must be a Vec type")
    
    inc.normalise()
    norm.normalise()
    
    A = inc.dot(norm)
    A2 = A * A
    mu = n1 / n2
    mu2 = mu * mu
    
    
    cp1 = norm.times( sign(A) * (1 - mu2 * (1 - A2)) ** 0.5 - (A * mu))
    cp2 = inc.times(mu)
    
    return cp1.add(cp2).normalise()



In [4]:
class Ray:
    def __init__(self, direction: Vec, start_pos: Vec, start_boundary = None, level = 0):
        
        if not isinstance(direction, Vec):
            raise TypeError("direction must be a Vec type")
        if not isinstance(start_pos, Vec):
            raise TypeError("start_pos must be a Vec type")
        
        self.direction = direction
        self.direction.normalise()
        
        self.start_pos = start_pos
        
        # If start_boundary = None, then treat the ray to be in air (n = 1)
        self.start_boundary = start_boundary
        
        # The number of interactions have happened before this ray's existance
        # Very first ray is level = 0, reflected or refracted is level = 1, next interaction is level = 2
        self.level = level
        
        self.end_boundary = None
        self.end_pos = None
        self.refracted_ray = None
        self.reflected_ray = None
        
    # Finds distance from start position given 1 coordinate from one of x, y or z
    def where(self, x = None, y = None, z = None):
        
        where_pos = np.array([x, y, z])
        val_arg = 0
        
        None_count = 0
        for i, val in enumerate(where_pos):
            if val is None:
                None_count+=1
            else:
                val_arg = i
                
        if None_count != 2:
            return None
            
            
        # Only 1 value to test
        
        
        if self.direction.vec[val_arg] == 0:
            return None
        
        dist = (where_pos[val_arg] - self.start_pos.vec[val_arg]) / self.direction.vec[val_arg]
        
        # Cant have points before the starting position
        if dist < 0:
            return None
        
        return dist
        
        
    def pos_from_distance(self, dist):
        return self.start_pos.add( self.direction.times(dist) )
    
    def pos_from_one_coord(self, **kwargs):
        dist = self.where(**kwargs)
        
        if dist is None:
            return None
        else:
            return self.pos_from_distance(dist)
        
    
    def update_ray(self, clipping_distance = CLIP_DIST):
        
        # Must have an end boundary (need to know norm and refractive index)
        if self.end_boundary is None:
            return False
        
        self.end_pos = self.end_boundary.find_intercept(self)
        
        # Must have an end position (None suggests it goes to inf)
        if self.end_pos is None:
            return False
        
        
        
        # Add a very small distance to stop ray clipping to boundaries
        refracted_pos = self.end_pos + clipping_distance * self.direction
        reflected_pos = self.end_pos - clipping_distance * self.direction
        
        # Initial refractive index
        n1 = 1
        if self.start_boundary is None:
            n1 = 1
        else:
            n1 = self.start_boundary.get_n()
        
        # The refractive index of the new medium 
        n2 = self.end_boundary.get_n()
        
        
        self.reflected_ray = Ray( 
            reflect(
                self.direction, 
                self.end_boundary.get_norm(self.end_pos)
            ),
            Vec(reflected_pos),
            # The ray is in the same medium
            start_boundary = self.start_boundary,
            level = self.level+1
        )
        
        self.refracted_ray = Ray( 
            refract(
                self.direction, 
                self.end_boundary.get_norm(self.end_pos),
                n1,
                n2
            ),
            Vec(refracted_pos),
            # The ray has entered the new medium
            start_boundary = self.end_boundary,
            level = self.level+1
        )
        
        return True

In [5]:
class Plane:
    def __init__(self, norm: Vec, pos: Vec):
        if not isinstance(norm, Vec):
            raise TypeError("norm must be a Vec type")
        if not isinstance(pos, Vec):
            raise TypeError("pos must be a Vec type")
        
        self.norm = norm
        self.norm.normalise()
        
        self.p0 = pos
        
    def ray_intercept_dist(self, ray: Ray, include_negative = False, parallel_case = False):
        if not isinstance(ray, Ray):
            raise TypeError("ray must be a Ray type")
        
        # d = n_p * (r_plane0 - r_ray0) / (n_plane * n_ray)
        
        dot = ray.direction.dot(self.norm.vec) 
        
        if abs(dot) < ERROR:
            # Ray is parallel
            if not parallel_case:
                return None
            else:
                return self.point_is_in(ray.start_pos)
        
        top_part = self.norm.dot(self.p0.add( - ray.start_pos.vec))
        
        d = top_part / dot
        
        
        if d < 0 and not include_negative:
            # Ray has passed plane already
            return None
        else:
            return d
        
    def point_is_in(self, point: Vec):
        if not isinstance(point, Vec):
            raise TypeError("point must be a Vec type")
        return ((self.p0 - point) * self.norm) > 0
    
     
class PlaneSurface:
    def __init__(self, *surfaces, n = 1):
        
        # Refractive index
        self.n = n
        self.surfaces = surfaces
        
    def find_intercept(self, ray: Ray):
        if not isinstance(ray, Ray):
            raise TypeError("ray must be a Ray type")
        
        surface_dist_intercept = []
        index = []
        
        for i, surface in enumerate(self.surfaces):
            d = surface.ray_intercept_dist(ray, include_negative = True, parallel_case = True)
            if d is None or d == False:
                return None
            elif d != True:
                # Never need to check plane if ray is parrallel and inside to plane
                index.append(i)
                surface_dist_intercept.append( d )
            
        surface_dist_intercept = np.array(surface_dist_intercept)
        index = np.array(index)
        
        
        argsort_dist = np.argsort(surface_dist_intercept)
        
        sorted_index = index[argsort_dist]
        
        sorted_dist = surface_dist_intercept[argsort_dist]
        
        
        # check which side ray lies at -inf
        # can take arbitary value away, choose 1
        d_inf = sorted_dist - 1
        
        ray_pos_inf = ray.pos_from_distance(d_inf)
        
        initial_state = np.full(len(sorted_index), True)
        
        for i, index in enumerate(sorted_index):
            initial_state[i] = self.surfaces[index].point_is_in(ray_pos_inf)
            
        
        in_states = np.full(True, shape=(len(index) + 1, len(index)))
        
        in_states[0] = initial_state
        
        for i in range(len(sorted_index)):
            for j in range(len(sorted_index)):
                if i + 1 == j:
                    in_states[i+1][j] = not in_states[i][j]
                else:
                    in_states[i+1][j] = in_states[i][j]
        
        which_surface = None
        
        
        for i in range(len(sorted_index)):
            if False not in in_states[i+1]:
                which_surface = i
                break
                
        if which_surface is None:
            return None
        else:
            entrance_index = sorted_index[which_surface]
            exit_index = sorted_index[which_surface + 1]
            
            if sorted_dist[which_surface] > 0:
                return self.surfaces[sorted_index[which_surface]], sorted_dist[which_surface]
            elif sorted_dist[which_surface + 1] > 0:
                return self.surfaces[sorted_index[which_surface + 1]], sorted_dist[which_surface + 1]
            else:
                return None
        


In [6]:
class Square(PlaneSurface):
    def __init__(self, center: Vec, length = 1, n = 1):
        
        if not isinstance(center, Vec):
            raise TypeError("center must be a Vec type")
        
        nums = [-1, 1]
        
        norms = []
        points = []
        
        for i in range(3):
            for j in nums:
                norm = np.zeros(3)
                norm[i] = j
                norms.append( Vec(norm) )
                
                point = center.vec.copy()
                point[i] = point[i] - (0.5 * length)
                points.append(point)
                
        surfaces = [Plane(Vec(norms[i]), Vec(points[i])) for i in range(6)]
                
                
        
        super().__init__(*surfaces, n = n)




In [7]:
class Scene:
    def __init__(self, *items):
        self.items = items
        
    def update_ray(self, ray: Ray):
        if not isinstance(ray, Vec):
            raise TypeError("ray must be a Vec type")
        
        nearest_dist = None
        
        for item in self.items:
            
            intercept_info = item.find_intercept(ray)
            
            if intercept_info is not None:
                
                if nearest_dist is None or intercept_info[1] < nearest_dist:
                    nearest_dist = intercept_info[1]
                    ray.end_boundary = intercept_info[0]
         
        
        ray.update_ray()
        
        

In [8]:
sq = Square(center = Vec(np.array([0, 0, 0])), n = 2)
ray = Ray(direction = Vec(np.array([1, 1, 1])), start_pos = Vec(np.array([-2, -2, -2])))
scene = Scene(sq)

scene.update_ray(ray)

TypeError: unsupported operand type(s) for *: 'Vec' and 'Vec'

array([3, 8])